In [10]:
# Initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
import csv
import json
import numpy as np
import dateparser
from Constants import Constants
from PreprocessingTools import CustomerNameCleaningFunctions
from PreprocessingTools import DateCleaningFunctions
from PreprocessingTools import SpecializedDateCleaningFunctions

pn.extension()

%matplotlib inline

In [11]:
# Build tools
debug_level = 0
constants = Constants()
tool_customer = CustomerNameCleaningFunctions(debug_level)
tool_date = DateCleaningFunctions(debug_level)
tool_special_date = SpecializedDateCleaningFunctions(debug_level)


In [12]:
# Read anonymized data
atlas           = pd.read_csv(constants.ANON_ATLAS_FILE_PATH, index_col="Customers")
forecast        = pd.read_csv(constants.ANON_FORECAST_DATA_FILE_PATH, index_col="Organization Name")
revenue2020     = pd.read_csv(constants.ANON_REVENUE2020_FILE_PATH, index_col="Name")
revenue2020A    = pd.read_csv(constants.ANON_REVENUE2020A_FILE_PATH, index_col="Payee Name")

In [18]:
tool_date.cleanup_date_string_list(atlas.iloc[0:5]["Invoice Date"])

Failed to parse: 3/20/2015
Failed to parse: 5/28/2015
Failed to parse: 6/23/2015
Failed to parse: 6/26/2015
Failed to parse: 10/7/2015


[]

In [13]:
# Clean up ATLAS data

## Invoice Date
# atlas["Invoice Date"]       = tool_date.cleanup_date_string_list(atlas["Invoice Date"])

## Invoice Amount
# atlas["Invoice Amount"]      = tool_date.cleanup_dollar_string_list(atlas["Invoice Amount"])

## Dates of service
# dates_of_service            = tool_special_date.extract_subscription_dates_list(atlas["Dates of service "])
# atlas["Service Start"]      = dates_of_service[0]
# atlas["Service End"]        = dates_of_service[1]
# atlas.drop(columns=["Dates of service "], inplace=True)

atlas.head()

,Invoice Date,Invoice #,Invoice Amount,Subscription,Account Code,Dates of service
Customers,,,,,,
University 1,3/20/2015,ATLAS 315,"$72,000.00",1 Year,4700-0-00-00000-18-0000,1 Year Subscription 3/18/15 to 6/30/16
University 102,5/28/2015,AJ501,"$3,500.00",1 Year,4700-0-00-00000-16-0000,1 Year Subscription 6/1/15 to 6/30/16
University 3,6/23/2015,AJ502,"$3,500.00",1 Year,4700-0-00-00000-17-0000,1 Year Subscription 6/10/15 to 6/30/16
University 4,6/26/2015,AJ503,"$6,500.00",1 Year,4700-0-00-00000-32-0000,1 Year Subscription 6/1/15 to 6/30/16
University 5,10/7/2015,AJ504,$750.00,1 Year,4700-0-00-00000-20-0000,1 Year Subscription 10/05/15 to 9/30/16
